In [1]:
config_manager_block = """
local FB_STATE = 0

local ECC_START = 0
local ECC_INIT = 1
local ECC_HOME = 2
local ECC_FEED = 3
local ECC_COLLECT = 4
local ECC_REJECT = 5
local ECC_READY = 6
local ECC_COLL_CONFIRM = 7
local ECC_FEED_CONFIRM = 8

local EI_INIT = 0
local EI_REQ_HOME = 1
local EI_REQ_FEED = 2
local EI_REQ_COLLECT = 3
local EI_REQ_REJECT = 4
local EI_REQ = 5
local EI_RESET = 6
local EI_CONFIRM = 7
local EO_INITO = 0
local EO_READY = 1
local EO_S5_GATE_CLOSE = 2
local EO_S5_GATE_OPEN = 3
local EO_CNF = 4
local EO_REQ_CONFIRMATION = 5

local DI_robotinoPos = 33554432
local DO_configurationID = 67108864


local function alg_INIT(fb)
  local STATE = fb[FB_STATE]
  local VAR_configurationID = fb[DOconfigurationID]
  VAR_configurationID = "NONE"
  fb[DOconfigurationID] = VAR_configurationID
end

local function alg_HOME(fb)
  local STATE = fb[FB_STATE]
  local VAR_configurationID = fb[DOconfigurationID]
  VAR_configurationID = "HOME"
  fb[DOconfigurationID] = VAR_configurationID
end

local function alg_FEED(fb)
  local STATE = fb[FB_STATE]
  local VAR_configurationID = fb[DOconfigurationID]
  VAR_configurationID = "FEED"
  fb[DOconfigurationID] = VAR_configurationID
end

local function alg_COLLECT(fb)
  local STATE = fb[FB_STATE]
  local VAR_configurationID = fb[DOconfigurationID]
  VAR_configurationID = "COLLECT"
  fb[DOconfigurationID] = VAR_configurationID
end

local function alg_REJECT(fb)
  local STATE = fb[FB_STATE]
  local VAR_configurationID = fb[DOconfigurationID]
  VAR_configurationID = "REJECT"
  fb[DOconfigurationID] = VAR_configurationID
end

local function enterECC_START(fb)
  fb[FB_STATE] = ECC_START
  return true
end

local function enterECC_INIT(fb)
  fb[FB_STATE] = ECC_INIT
  alg_INIT(fb)
  fb(EOINITO)
  return true
end

local function enterECC_HOME(fb)
  fb[FB_STATE] = ECC_HOME
  fb(EOS5_GATE_CLOSE)
  alg_HOME(fb)
  fb(EOCNF)
  return true
end

local function enterECC_FEED(fb)
  fb[FB_STATE] = ECC_FEED
  fb(EOS5_GATE_CLOSE)
  alg_FEED(fb)
  fb(EOCNF)
  return true
end

local function enterECC_COLLECT(fb)
  fb[FB_STATE] = ECC_COLLECT
  fb(EOS5_GATE_OPEN)
  alg_COLLECT(fb)
  fb(EOCNF)
  return true
end

local function enterECC_REJECT(fb)
  fb[FB_STATE] = ECC_REJECT
  fb(EOS5_GATE_CLOSE)
  alg_REJECT(fb)
  return true
end

local function enterECC_READY(fb)
  fb[FB_STATE] = ECC_READY
  fb(EOREADY)
  return true
end

local function enterECC_COLL_CONFIRM(fb)
  fb[FB_STATE] = ECC_COLL_CONFIRM
  fb(EOREQ_CONFIRMATION)
  return true
end

local function enterECC_FEED_CONFIRM(fb)
  fb[FB_STATE] = ECC_FEED_CONFIRM
  fb(EOREQ_CONFIRMATION)
  return true
end

local function transition(fb, id)
  local STATE = fb[FB_STATE]
  local VAR_robotinoPos = fb[DIrobotinoPos]
  local VAR_configurationID = fb[DOconfigurationID]
  if ECC_START == STATE then
    if EI_INIT == id and true then return enterECC_INIT(fb)
    else return false
    end
  elseif ECC_INIT == STATE then
    if true and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_HOME == STATE then
    if true and (VAR_robotinoPos == "HOME") then return enterECC_READY(fb)
    elseif EI_RESET == id and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_FEED == STATE then
    if true and (VAR_robotinoPos == "FEED") then return enterECC_FEED_CONFIRM(fb)
    elseif EI_RESET == id and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_COLLECT == STATE then
    if true and ((VAR_robotinoPos == "COLLECT") or (VAR_robotinoPos == "COLL")) then return enterECC_COLL_CONFIRM(fb)
    elseif EI_RESET == id and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_REJECT == STATE then
    if true and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_READY == STATE then
    if EI_REQ_HOME == id and true then return enterECC_HOME(fb)
    elseif EI_REQ_COLLECT == id and true then return enterECC_COLLECT(fb)
    elseif EI_REQ_REJECT == id and true then return enterECC_REJECT(fb)
    elseif EI_REQ_FEED == id and true then return enterECC_FEED(fb)
    else return false
    end
  elseif ECC_COLL_CONFIRM == STATE then
    if EI_CONFIRM == id and true then return enterECC_READY(fb)
    elseif EI_RESET == id and true then return enterECC_READY(fb)
    else return false
    end
  elseif ECC_FEED_CONFIRM == STATE then
    if EI_CONFIRM == id and true then return enterECC_READY(fb)
    elseif EI_RESET == id and true then return enterECC_READY(fb)
    else return false
    end
  else return false
  end
end

local function executeEvent(fb, id)
  local modified = transition(fb, id)
  while modified do
    modified = transition(fb, -1)
  end
end

local interfaceSpec = {
  numEIs = 8,
  EINames = {"INIT", "REQ_HOME", "REQ_FEED", "REQ_COLLECT", "REQ_REJECT", "REQ", "RESET", "CONFIRM"},
  EIWith = {255, 255, 255, 255, 255, 0, 255, 255, 255},
  EIWithIndexes = {0, 0, 0, 0, 0, 1, 0, 0},
  numEOs = 6,
  EONames = {"INITO", "READY", "S5_GATE_CLOSE", "S5_GATE_OPEN", "CNF", "REQ_CONFIRMATION"},
  EOWith = {255, 0, 255, 255, 255, 0, 255, 255},
  EOWithIndexes = {0, 1, 0, 0, 1, 0},
  numDIs = 1,
  DINames = {"robotinoPos"},
  DITypeNames = {"STRING"},
  numDOs = 1,
  DONames = {"configurationID"},
  DOTypeNames = {"STRING"},
  numAdapters = 0,
  adapterInstanceDefinition = {
  }
}

local internalVarsInformation = {
  numIntVars = 1,
  intVarsNames = {},
  intVarsDataTypeNames = {},
}

return {ECC = executeEvent, interfaceSpec = interfaceSpec, internalVarsInformation = internalVarsInformation}
"""

config_manager_composite_block = """
local EI_INIT = 0
local EI_REQ_HOME = 1
local EI_REQ_FEED = 2
local EI_REQ_COLLECT = 3
local EI_REQ_REJECT = 4
local EI_RESET = 5
local EI_CONFIRM = 6
local EO_INITO = 0
local EO_READY = 1
local EO_S5_GATE_CLOSE = 2
local EO_S5_GATE_OPEN = 3
local EO_REQ_CONFIRMATION = 4

local DO_configurationID = 67108864

local interfaceSpec = {
  numEIs = 7,
  EINames = {"INIT", "REQ_HOME", "REQ_FEED", "REQ_COLLECT", "REQ_REJECT", "RESET", "CONFIRM"},
  EIWith = {255, 255, 255, 255, 255, 255, 255},
  EIWithIndexes = {0, 0, 0, 0, 0, 0, 0},
  numEOs = 5,
  EONames = {"INITO", "READY", "S5_GATE_CLOSE", "S5_GATE_OPEN", "REQ_CONFIRMATION"},
  EOWith = {255, 0, 255, 255, 255, 0, 255},
  EOWithIndexes = {0, 1, 0, 0, 1},
  numDIs = 0,
  DINames = {},
  DITypeNames = {},
  numDOs = 1,
  DONames = {"configurationID"},
  DOTypeNames = {"STRING"},
  numAdapters = 0,
  adapterInstanceDefinition = {
  }
}

local fbnSpec = {
  internalFBs = {
    {fbNameID = "FB1", fbTypeID = "ConfigurationManager"},
    {fbNameID = "ROBOTINO_ACTION", fbTypeID = "PUBLISH_1"},
    {fbNameID = "ROBOTINO_STATE", fbTypeID = "SUBSCRIBE_1"}
  },
  parameters = {
  },
  eventConnections = {
    {srcID = "REQ_FEED", srcFBNum = -1, dstID = "REQ_FEED", dstFBNum = 0},
    {srcID = "IND", srcFBNum = 2, dstID = "REQ", dstFBNum = 0},
    {srcID = "CONFIRM", srcFBNum = -1, dstID = "CONFIRM", dstFBNum = 0},
    {srcID = "REQ_COLLECT", srcFBNum = -1, dstID = "REQ_COLLECT", dstFBNum = 0},
    {srcID = "CNF", srcFBNum = 0, dstID = "REQ", dstFBNum = 1},
    {srcID = "REQ_CONFIRMATION", srcFBNum = 0, dstID = "REQ_CONFIRMATION", dstFBNum = -1},
    {srcID = "RESET", srcFBNum = -1, dstID = "RESET", dstFBNum = 0},
    {srcID = "REQ_HOME", srcFBNum = -1, dstID = "REQ_HOME", dstFBNum = 0},
    {srcID = "INITO", srcFBNum = 0, dstID = "INITO", dstFBNum = -1},
    {srcID = "INITO", srcFBNum = 1, dstID = "INIT", dstFBNum = 0},
    {srcID = "INIT", srcFBNum = -1, dstID = "INIT", dstFBNum = 2},
    {srcID = "S5_GATE_OPEN", srcFBNum = 0, dstID = "S5_GATE_OPEN", dstFBNum = -1},
    {srcID = "READY", srcFBNum = 0, dstID = "READY", dstFBNum = -1},
    {srcID = "REQ_REJECT", srcFBNum = -1, dstID = "REQ_REJECT", dstFBNum = 0},
    {srcID = "INITO", srcFBNum = 2, dstID = "INIT", dstFBNum = 1},
    {srcID = "S5_GATE_CLOSE", srcFBNum = 0, dstID = "S5_GATE_CLOSE", dstFBNum = -1}
  },
  fannedOutEventConnecitons = {

  },
  dataConnections = {
    {srcID = "configurationID", srcFBNum = 0, dstID = "configurationID", dstFBNum = -1},
    {srcID = "RD_1", srcFBNum = 2, dstID = "robotinoPos", dstFBNum = 0},
    {srcID = "configurationID", srcFBNum = 0, dstID = "SD_1", dstFBNum = 1}
  },
  fannedOutDataConnecitons = {
  },
  numFBs = 3,
  numECons = 16,
  numFECons = 0,
  numDCons = 3,
  numDECons = 0,
  numParams = 0
}

return {interfaceSpec = interfaceSpec, fbnSpec = fbnSpec}
"""

In [2]:
import socket
import sys
import time

In [3]:
device_port = 61499

In [4]:
class DeviceConnection():
    
    def __init__(self, port):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect(('localhost', port))
        self.id = 0
   
    def send(self, resource, action, body):
        content = '<Request ID="' + str(self.id) + '" Action="' + action + '">' + body + '</Request>'
        self.id = self.id + 1
        encoded = ('\x50\x00'+ chr(len(resource)) + resource + '\x50\x00' + chr(len(content)) + content).encode('ascii')
        self.socket.sendall(encoded)
        print(self.socket.recv(1024)[3:].decode('ascii'))
    
    def close(self):
        self.socket.close()
    

In [ ]:
connection = DeviceConnection(device_port)

In [ ]:
connection.send('', 'CREATE', '<FB Name="Blinky_RES" Type="EMB_RES" />')

In [ ]:
connection.send('', 'CREATE', '<FBType Name="ConfigurationManager">' + config_mananager_block + '</FBType>')

In [ ]:
connection.send('', 'CREATE', '<FBType Name="ConfigurationManagerC">' + config_manager_composite_block + '</FBType>')

In [ ]:
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_CYCLE" Type="E_CYCLE" />')
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_SWITCH" Type="E_SWITCH" />')
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_SR" Type="E_SR" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="START.COLD" Type="E_CYCLE.START" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="START.WARM" Type="E_CYCLE.START" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_CYCLE.EO" Type="E_SWITCH.EI" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SWITCH.EO0" Type="E_SR.S" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SWITCH.EO1" Type="E_SR.R" />')
connection.send('Blinky_RES', 'WRITE', '<Connection Source="T#1s" Type="E_CYCLE.DT" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SR.Q" Type="E_SWITCH.G" />')

In [ ]:
connection.send('Blinky_RES', 'START', '')

In [ ]:
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_SWITCH.EO0" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_CYCLE.EO" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_CYCLE.E_DELAY.EO" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_SR.Q" Destination="*" />')

In [ ]:
for i in range(100):
    connection.send('', 'READ', '<Watches/>')
    time.sleep(0.1)

In [ ]:
connection.close()

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(('239.0.0.1', 61000))

data = s.recvfrom(16)
data

In [ ]:
import struct

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('', 61000))

mreq = struct.pack("=4sl", socket.inet_aton('239.0.0.1'), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

while True:
    print([i for i in sock.recv(10240)])